<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Symbol-to-Symbol_Attack_Clean-1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbol-to_Symbol Attack

# Loading Necesary Libraries

In [48]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

# Generate Train/Test data

In [49]:
def create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  X=None
  Y=[]
  colnames=['i'+str(i) for i in range(nsymbols)]

  for d in distances:
    dist=d*span_length
    if dist<min_dist or dist>max_dist: continue
    filename='consts_'+str(d)+'span.csv'
    df_aux=pd.read_csv(input_data_path+'/'+filename, sep=",", header=None)
    df_aux = df_aux.T
    df_aux.columns=colnames
    Y=Y+[dist]*df_aux.shape[0]
    if X is None: X=df_aux
    else: X=X.append(df_aux)
  return X, Y

In [50]:
def strToTuple(s):
    s_aux=s.split("i")
    s=s_aux[0]+"j"
    return complex(s)

In [51]:
def test_train_indexes_v1():
  train_idxs = []
  test_idxs = []
  for i in range(25):
      for j in range(25):
          train_idxs.append(50*i + j)
          test_idxs.append(50*(i+1)-1-j)
  return train_idxs, test_idxs

In [52]:
def test_train_indexes_v2():
  train_idxs2 = []
  test_idxs2 = []
  for i in range(25):
      for j in range(25):
          test_idxs2.append(50*(i+1)-1-j)

  for i in range(25):
    for j in range(2):
      train_idxs2.append(50*i + j)
  return train_idxs2, test_idxs2

# Utils

## Gradient Descent Function

In [53]:
def grad_b (M,m_og,m_tg,b,beta):
  return 2*beta*(M @ m_og + b - m_tg)

In [54]:
def grad_M(M,cov_og,cov_tg,alpha):
    error = M @ cov_og @ np.transpose(M) - cov_tg  

    m1 = (2*error[0][0] * (2*cov_og[0][0]*M[0][0] + 2*cov_og[0][1]*M[0][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[1][0] + cov_og[0][1]*M[1][1]))
    
    m2 = (2*error[0][0] * (2*cov_og[1][1]*M[0][1] + 2*cov_og[0][1]*M[0][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[1][0] + cov_og[1][1]*M[1][1]))
    
    m3 = (2*error[1][1] * (2*cov_og[0][0]*M[1][0] + 2*cov_og[0][1]*M[1][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[0][0] + cov_og[0][1]*M[0][1]))

    m4 = (2*error[1][1] * (2*cov_og[0][0]*M[1][1] + 2*cov_og[0][1]*M[1][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[0][0] + cov_og[1][1]*M[0][1]))
    
    return alpha*np.array([[m1, m2], [m3, m4]])

In [55]:
def gradient_descent(alpha,beta,m_tg,m_og,cov_tg,cov_og,nu,log):
  Ms = []
  bs = []
  M = np.random.rand(2,2)
  #M = np.array([[1, 0], [0, 1]])
  b = np.random.rand(2,1)
  #b = np.array([[0], [0]])
  #for i in range(100000):
  i = 0
  while True:
    # print('From:', M @ cov_source @ np.transpose(M))
    # print('To:', cov_target)
    L = loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b) 
    if  L < 1e-20:
        Ms.append(M)
        bs.append(b)
        break
    b = b - nu*grad_b(M,m_og,m_tg,b,beta)
    M = M - nu*grad_M(M,cov_og,cov_tg,alpha)
    i+= 1
    if (i>= 5000 and i <= 5025) or (i>= 5975 and i <= 6000):
      Ms.append(M)
      bs.append(b)
    if not i%5000 and log: print(L)
  if log: print("-"*25)
  return Ms,bs

## Compute parameters and mean/covariance

In [56]:
def compute_mean_and_cov(data):
  aux_x = [] # Reales
  aux_y = [] # Imag
  for obs in data:
    aux_x.append(obs[0])
    aux_y.append(obs[1])
  return np.array([[np.mean(aux_x)],[np.mean(aux_y)]]), np.cov(aux_x,aux_y)


def total_loss(m_mod,m_tg,cov_mod,cov_tg):
  return (sum(sum(np.power(cov_mod-cov_tg,2))) + sum(np.power(m_mod - m_tg,2)))[0]


def loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b):
  first = alpha*sum(sum(np.power(M @ cov_og @ np.transpose(M) - cov_tg, 2)))
  second = beta*sum(np.power(M @ m_og + b - m_tg, 2))
  a = first+second
  return a

In [84]:
def compute_parameters (const,method,source,target, params = None, log = False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  q_target = []

  for i in range(len(source)):

    q_source += [[x.real,x.imag] for x in source.values.tolist()[i] if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]
    q_target += [[x.real,x.imag] for x in target.values.tolist()[i]if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]

  unbiased = (len(q_source)/(len(q_source)-1) * 125/126)

  mean_source,cov_source = compute_mean_and_cov(q_source)
  mean_target, cov_target= compute_mean_and_cov(q_target)

  cov_source *= unbiased
  cov_target *= unbiased

  if method == "GD":
  
    alpha = params["alpha"] if params["alpha"] is not None and params["alpha"]>0 and params["alpha"]<=1 and params["alpha"]>params["beta"]  else 3/4
    beta = params["beta"] if params["beta"] is not None and params["beta"]>=0 and params["beta"]<1 and params["beta"]<params["alpha"]  else 1/4

    M, b = gradient_descent(alpha,beta,mean_target,mean_source,cov_target,cov_source, 0.5,log)
    return M,b

  elif method == "Z":
    return mean_source,cov_source,mean_target,cov_target

## Modify GD/Z

In [58]:
def modify_const_GD(const,source,M,b, target = None, return_plots=False):

  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]
      
  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  for mult in mults:
    res = (M @ mult + b).tolist()

    # --------- Old line ---------
    #new_points.append([res[0][0],res[1][0]])
    nested_check = any(isinstance(i, list) for i in res[0])
    if nested_check:
      new_points.append([res[0][0][0],res[0][1][0]])
    else:
      new_points.append([res[0][0],res[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      return new_points, q_target

  return new_points, indexes

In [59]:
def modify_const_Z(const,source,mean_source,cov_source,mean_target,cov_target,target = None, return_plots=False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]

  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  w, v = np.linalg.eig(cov_source)
  S1 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)
  w, v = np.linalg.eig(cov_target)
  S2 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)

  for mult in mults:
    normalized = np.linalg.inv(S1) @ (mult - mean_source)
    denormalized = S2 @ normalized + mean_target

    new_points.append([denormalized[0][0], denormalized[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      

  return new_points, indexes

## Compare plots

In [60]:
def generate_comparison_plot(mod_points,ptarget, return_plots=False):
  x1 = [x[0] for  x in mod_points]
  y1 = [x[1] for  x in mod_points]
  x2 = [x[0] for  x in ptarget]
  y2 = [x[1] for  x in ptarget]

  if return_plots:
      return go.Scatter(x = x1, y = y1, mode='markers'), go.Scatter(x = x2, y = y2,mode='markers') 

  fig = make_subplots(rows=1, cols=2)
  fig.add_trace(
    go.Scatter(x = x1, y = y1,mode='markers'),
    
    row=1, col=1
  )
  fig.add_trace(
    go.Scatter(x = x2, y = y2,mode='markers'),
    row=1, col=2
  )
  fig.update_layout(height=500, width=1000, title_text="Point comparison",autosize = False)
  fig.show()

## Distance Calc

In [61]:
def L2dist(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

# Main

In [121]:
def mod_df_generator(df):
  distance_arr = []
  sample_arr = []
  symbol_arr = []
  real_arr = []
  imag_arr = []
  
  distance_cnt = 80
  distance_multiplier = 2

  sample_cnt = 0
  for i in range(len(df)):
    symbol_cnt = 1
    for x in df.values.tolist()[i]:
      real_arr.append(x.real)
      imag_arr.append(x.imag)

      sample_arr.append(i+1)
      symbol_arr.append(symbol_cnt)

      distance_arr.append(distance_cnt)
      symbol_cnt += 1

    if (i+1) % 50 == 0:
      distance_cnt = distance_multiplier * distance_cnt
      distance_multiplier += 1
      break
      
  
  data = {'Distance_Id':distance_arr,'Sample_Id':sample_arr, 'Symbol_Id':symbol_arr, 'Real':real_arr, 'Imag':imag_arr}

  mod_df = pd.DataFrame(data)
  
  return mod_df

In [86]:
# Obtain df for each distance (X) and  distance*nsymbols (Y)
input_data_path=path+"/rawData"
distances=[i for i in range(1,26)]
X, Y = create_df(input_data_path, distances, nsymbols = 2048, min_dist = 0, max_dist = 3000)
X = X.applymap(strToTuple)

In [126]:
print(range(len(X)))
for i in range(len(X)):
  print(i)
  break

range(0, 1250)
0


In [94]:
print(X.shape)

(1250, 2048)


In [95]:
print(distances)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [129]:
display(X.head(50))

,i0,i1,i2,i3,i4,i5,i6,i7,i8,i9,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,-3.102503+2.897472j,-3.085198+2.984675j,-3.162588+2.979846j,1.078790-3.042417j,2.987543+2.939280j,-3.037535-1.144484j,3.084602-2.839832j,1.035293+0.941187j,-2.974975-0.968042j,-3.060942-3.061597j,...,1.004077-1.057959j,2.995127-0.926751j,1.064111+1.025560j,3.074374-0.843430j,2.896799+3.103109j,-2.929350-3.040434j,0.979544+3.056479j,-1.000231-3.017304j,-3.107024+2.907814j,2.967643+3.060832j
1,-3.107614+2.882100j,-3.195330+2.958744j,-3.138298+3.005035j,1.068618-2.953400j,2.929212+2.961882j,-2.914321-1.104195j,3.015965-2.879752j,0.998438+0.992891j,-2.956116-1.055437j,-3.018871-3.076542j,...,1.091429-0.974361j,2.992922-1.016159j,0.985157+0.990884j,2.957722-0.935638j,2.916176+3.072065j,-2.934550-2.993022j,0.895364+3.003775j,-0.863137-2.949139j,-3.027753+3.105827j,3.013879+3.173615j
2,-3.102044+2.996718j,-3.088564+2.932023j,-3.044919+2.886925j,1.088239-2.987504j,2.906977+2.975853j,-2.981455-1.094215j,3.087710-2.941174j,0.963332+0.964856j,-2.910665-1.101440j,-3.031983-2.955348j,...,0.907715-1.024220j,3.035462-1.023603j,0.893086+0.966805j,2.973861-0.964295j,2.885995+3.118869j,-2.902216-3.070452j,0.950042+2.991643j,-0.923820-3.019035j,-3.117300+2.983768j,2.911937+3.036795j
3,-3.094574+2.838886j,-3.084941+2.883885j,-3.206773+2.913172j,1.167390-2.998481j,2.946582+3.008025j,-2.875261-1.114056j,3.082594-2.919445j,1.062736+1.109840j,-3.032694-1.023069j,-3.014736-2.995780j,...,1.007585-0.965973j,3.109686-0.916238j,0.938951+1.009892j,2.944321-0.931511j,2.984951+3.224162j,-2.980087-3.177183j,1.074162+3.000073j,-0.881409-2.994283j,-3.101536+3.036561j,3.009649+3.097725j
4,-3.131272+2.975436j,-3.147729+2.892179j,-3.216454+2.970119j,1.113168-2.977314j,2.735244+3.059486j,-2.972129-1.059037j,3.077705-2.907277j,0.982013+0.963790j,-2.979806-1.099596j,-2.966601-3.127786j,...,0.947200-0.974594j,2.976825-0.881520j,0.988876+1.038691j,3.002536-0.849527j,2.939834+3.000328j,-2.817801-3.031001j,0.810747+3.078971j,-0.891457-3.071797j,-3.061338+3.016474j,2.975035+3.132928j
5,-3.090438+2.973794j,-3.073706+2.867736j,-3.140852+2.898120j,1.187309-3.014463j,2.839390+3.005876j,-2.886874-1.073987j,3.139224-2.981620j,1.006494+0.942777j,-2.980175-1.094746j,-2.945682-3.103147j,...,0.998216-0.988400j,2.963264-0.879207j,1.029979+0.920656j,2.943123-0.883128j,2.964480+3.135630j,-2.852127-3.119664j,0.899194+3.024696j,-0.918842-3.007361j,-3.075334+2.969879j,2.799566+3.236509j
6,-3.067704+2.961860j,-3.046359+2.945558j,-3.112837+2.950418j,1.085795-2.908489j,2.973876+3.035184j,-2.992605-1.003552j,3.113532-2.892502j,0.977231+1.045437j,-3.009415-1.082581j,-2.982919-3.057349j,...,1.023359-0.982491j,2.988181-0.863141j,1.031179+0.928928j,3.041440-0.934211j,2.850815+3.142959j,-2.908349-3.041781j,0.917171+2.950879j,-0.963712-3.049435j,-3.175106+3.096494j,2.943607+3.121697j
7,-3.037878+2.827476j,-3.090263+2.969087j,-3.154008+2.993321j,1.030608-2.916395j,2.906498+3.133124j,-2.971314-1.010123j,2.942582-2.903418j,0.943279+1.102930j,-2.863760-1.106864j,-2.938777-3.070959j,...,1.065370-1.037020j,2.984718-1.000431j,1.059136+1.055955j,3.064609-0.962689j,2.990571+3.174185j,-2.828426-3.144315j,0.956094+3.038083j,-0.965682-2.986870j,-3.060420+3.063362j,2.872580+3.134602j
8,-3.089017+2.918461j,-3.084965+2.948771j,-3.195042+2.967253j,1.146529-2.907999j,2.845664+3.003510j,-2.913547-1.068221j,2.961085-2.839242j,0.980456+0.892957j,-2.996532-1.191946j,-2.931919-3.075896j,...,0.985231-0.975949j,3.044131-0.954791j,1.065720+1.160877j,3.101998-0.945075j,2.982246+3.133776j,-2.969630-3.148216j,0.929499+3.077496j,-0.962507-2.975830j,-3.083632+2.938972j,3.053324+3.169537j
9,-2.978940+2.984961j,-2.990800+2.954478j,-3.095420+2.879122j,1.059433-3.052102j,2.927438+3.134012j,-2.959492-1.181712j,3.108569-3.074005j,0.903077+1.009141j,-3.022546-1.054651j,-3.023271-3.050276j,...,1.070948-1.008751j,2.987396-1.015533j,1.022064+1.015615j,2.966776-0.940106j,2.965667+3.002695j,-2.988324-3.116884j,0.973503+3.111471j,-0.868272-3.03

In [75]:
def symbol2symbol_main(method, quadrants, input_data_path, distances, output_filename, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  
  # Get input data
  X, Y = create_df(input_data_path, distances, nsymbols, min_dist, max_dist)
  X = X.applymap(strToTuple)
  
'''
  train_idxs2, test_idxs2 = test_train_indexes_v2()

  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True") 

  data_new = {}

  for i in range(0,25):
    data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}  

  first = True

  for i in tqdm(range(25)):
      print('\n')
      print("i = ",i)
      for j in range(i+1,25):
          print(" j = ",j)
          source = data_new[str(distances[i]*80)]['Train']
          target = data_new[str(distances[j]*80)]['Train']
          source_test = data_new[str(distances[i]*80)]['Test']

          source_test2 = source_test.copy()

          for quadrant in quadrants:
              print("   quadrant = ", quadrant)
              if method == "GD":
                M, b = compute_parameters(quadrant, 'GD', source, target, params=params)
                new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
                break
              elif method == "Z":
                mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target, params=None)
                new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)
                break
          break
      break

              for k in range(len(indexes)):
                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])
          
          F=[]
          for k in range(source_test2.shape[0]):
              data2=list(source_test2.iloc[k,:])
              data2=[[float(d.real), float(d.imag)] for d in data2]
              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
              mus=gmm.means_
              sigmas=gmm.covariances_
              features=[distances[j]*80]
              for z in selCP_pos:
                  mindist=None
                  k_inc=None
                  for w in range(16):
                      d=L2dist(mus[w],z)
                      if mindist is None or mindist>d:
                          mindist=d
                          k_inc=w
                  #print(k_inc)
                  #print(mus[k_inc])
                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                  #features = [*features, *mus[k_inc], *[covmat[0], covmat[3]]]
                  features = [*features, *mus[k_inc], *covmat]
              F.append(features)

          header=['dist']
          
          for j in selCP:
              header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
              #header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ii_'+str(j)]]
          with open(path+'/modifiedData/'+ method + output_filename, 'a', encoding='UTF8', newline='') as f:
              writer = csv.writer(f)
              # write the header
              if first:
                writer.writerow(header)
                first = False
              # write multiple rows
              writer.writerows(F)
'''

"\n              for k in range(len(indexes)):\n                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])\n          \n          F=[]\n          for k in range(source_test2.shape[0]):\n              data2=list(source_test2.iloc[k,:])\n              data2=[[float(d.real), float(d.imag)] for d in data2]\n              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)\n              mus=gmm.means_\n              sigmas=gmm.covariances_\n              features=[distances[j]*80]\n              for z in selCP_pos:\n                  mindist=None\n                  k_inc=None\n                  for w in range(16):\n                      d=L2dist(mus[w],z)\n                      if mindist is None or mindist>d:\n                          mindist=d\n                          k_inc=w\n                  #print(k_inc)\n                  #print(mus[k_inc])\n                  covmat=np.concatenate(list(sigmas

#Run Main 

## Load Enviroment

In [63]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [64]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

## Global Variables

In [65]:
# Global variables

distances=[i for i in range(1,26)]
nsamples=50
span_length=80
nsymbols=2048

######
min_dist=0
max_dist=3000
selCP=[1,7,10,15]
selCP_pos=[(-3,3),(1,1),(-1,-1),(1,-3)]
my_centers=[[-3,3],[-1,3],[1,3],[3,3],[-3,1],[-1,1],[1,1],[3,1],[-3,-1],[-1,-1],[1,-1],[3,-1], [-3,-3],[-1,-3],[1,-3],[3,-3]]
######

## Execution Z Score

### User inputs

In [66]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "Z" # or method = "GD"
quadrants = [1, 7, 10, 15]

# PLEASE spicify the name of the new file
# Output features
output_filename = "/Z_alpha_1.csv"

#Output constellation
#add later


# PLEASE review that the path below is correct
input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

filepath selected: /content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset/rawData


### Lauch

In [67]:
symbol2symbol_main(method, quadrants, input_data_path, distances, output_filename, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000)

  0%|          | 0/25 [00:00<?, ?it/s]



i =  0
 j =  1
   quadrant =  1


  0%|          | 0/25 [00:00<?, ?it/s]


## Execution Gradient Descent

### User Inputs

In [80]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "GD" # or method = "Z"
quadrants = [1, 7, 10, 15]

# PLEASE spicify the name of the new file
output_filename = "/GD_alpha_1.csv"

# PLEASE review that the path below is correct
input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

# Params for gradient descent alpha beta
a = 0.7
b = 0.2
params = {"alpha": float(a), "beta": float(b)}

filepath selected: /content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset/rawData


### Lauch

In [83]:
symbol2symbol_main(method, quadrants, input_data_path, distances, output_filename, params = params, nsymbols = 2048, min_dist = 0, max_dist = 3000)

  0%|          | 0/25 [00:00<?, ?it/s]



i =  0
 j =  1
   quadrant =  1
{'alpha': 0.7, 'beta': 0.2}
0.7
0.2


  0%|          | 0/25 [00:20<?, ?it/s]
